In [1]:
import sys, os
sys.path.insert(0,'/global/homes/b/bpb/repos/metatlas/' )
# sys.path.insert(1,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
# sys.path.insert(1,'/global/project/projectdirs/metatlas/projects/metatlas' )

from metatlas.helpers import dill2plots as dp

from metatlas.helpers import metatlas_get_data_helper_fun as ma_data
from metatlas import metatlas_objects as metob
# from metatlas.helpers import rt_corrector as rt_corrector
from ipywidgets import interact, interactive, fixed

import ipywidgets as widgets
from IPython.display import display

import json
import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


%matplotlib notebook

('NERSC=', True)


In [2]:
cids = metob.retrieve('Compounds',inchi_key = 'YGSDEFSMJLZEOE-UHFFFAOYSA-N')

/global/common/software/m2650/python-cori/lib/python2.7/site-packages/pymysql/cursors.py:166: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


In [12]:
class MS1Data(tables.IsDescription):
    mz = tables.Float32Col(pos=0)
    i = tables.Float32Col(pos=1)
    rt = tables.Float32Col(pos=2)


class MS2Data(MS1Data):
    # the rest are only relevant for ms2 spectra
    precursor_MZ = tables.Float32Col(pos=5)
    precursor_intensity = tables.Float32Col(pos=6)
    collision_energy = tables.Float32Col(pos=7)


class ScanInfo(tables.IsDescription):
    rt = tables.Float32Col(pos=0)
    polarity = tables.Int16Col(pos=1)
    ms_level = tables.Int16Col(pos=2)
    max_mz = tables.Float32Col(pos=3)
    min_mz = tables.Float32Col(pos=4)
    # the rest are only relevant for ms2 spectra
    precursor_MZ = tables.Float32Col(pos=5)
    precursor_intensity = tables.Float32Col(pos=6)
    collision_energy = tables.Float32Col(pos=7)


def read_spectrum(spectrum, index):
    """Read a single spectrum

    Parameters
    ----------
    spectrum : mzML spectrum
        The mzML spectrum to parse.

    Returns
    -------
    out : list of tuples
        List of values associated with each peak in the spectrum.
    """
    polarity = 'MS:1000130' in spectrum
    ms_level = spectrum['ms level']
    rt, units = spectrum['MS:1000016']
    if units != 'minute':
        rt /= 60

    collision_energy = spectrum.get('MS:1000045', 0)
    precursor_intensity = spectrum.get('MS:1000042', 0)
    precursor_MZ = spectrum.get('MS:1000744', 0)
    min_mz = spectrum.get('lowest m/z value', 0)
    max_mz = spectrum.get('highest m/z value', 0)

    info = (rt, polarity, ms_level, min_mz, max_mz, precursor_MZ,
            precursor_intensity, collision_energy)

    if ms_level == 1:
        data = [(mz, i, rt) for (mz, i) in spectrum.peaks]
    else:
        data = [(mz, i, rt, precursor_MZ, precursor_intensity,
                 collision_energy) for (mz, i) in spectrum.peaks]

    return data, info

extraAccessions = [
    ('MS:1000016', ['value', 'unitName']),  # scan start time
    ('MS:1000129', ['name']),  # negative scan
    ('MS:1000130', ['name']),  # positive scan
    ('MS:1000744', ['value']),  # selected ion m/z
    ('MS:1000042', ['value']),  # peak intensity
    ('MS:1000045', ['value']),  # collision energy
    ('MS:1000528', ['value']),  # lowest observed m/z
    ('MS:1000527', ['value']),  # highest observed m/z
    ('MS:1000529', ['value']),  # serial number
]

mzml_reader = Reader(in_file_name,extraAccessions=extraAccessions)


    info_table = out_file.create_table('/', 'info', description=ScanInfo)

    got_first = False

    for (ind, spectrum) in enumerate(mzml_reader):

        if got_first and spectrum['id'] == 1:
            # check for a repeat
            break
        try:
            data, info = read_spectrum(spectrum, ind)
        except (KeyError, TypeError):
            continue
        except Exception as e:
            sys.stdout.write('Read spectrum error: %s\n' % e)
            sys.stdout.flush()
            continue

        if not data:
            continue

        got_first = True

        if info[2] == 1:  # ms level
            if not info[1]:  # polarity
                table = ms1_neg
            else:
                table = ms1_pos
        elif not info[1]:
            table = ms2_neg
        else:
            table = ms2_pos

        table.append(data)
        table.flush

        info_table.append([info])

        if debug and not (ind % 100):
            sys.stdout.write('.')
            sys.stdout.flush()

    info_table.flush()

    for name in ['ms1_neg', 'ms2_neg', 'ms1_pos', 'ms2_pos']:
        table = out_file.get_node('/' + name)
        table.cols.mz.create_csindex()
        table.copy(sortby='mz', newname=name + '_mz')
        table.cols.mz.remove_index()




['nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',


In [3]:
for c in cids:
    print c.name, c.username

Salicylate///Bismuth Subsalicylate///Salicylate-sodium///Q4499058///SALICYLATE///Salicylic acid///salicylate///Sodium salicylate///Trolamine salicylate///salicylic acid///Magnesium salicylate///Trisalicylate-choline bpb
Salicylate///Bismuth Subsalicylate///Salicylate-sodium///Q4499058///SALICYLATE///Salicylic acid///salicylate///Sodium salicylate///Trolamine salicylate///salicylic acid///Magnesium salicylate///Trisalicylate-choline bpb
Salicylate///Bismuth Subsalicylate///Salicylate-sodium///Q4499058///SALICYLATE///Salicylic acid///salicylate///Sodium salicylate///Trolamine salicylate///salicylic acid///Magnesium salicylate///Trisalicylate-choline bpb


In [3]:
cids = metob.retrieve('FragmentationReferences',username='*',unique_id='b90ba793c41d44de8fc362f47811359e')
cids

[{'collision_energy': u'0',
  'creation_time': '2016-06-22T10:37:26',
  'description': u'No description',
  'enabled': True,
  'head_id': u'b90ba793c41d44de8fc362f47811359e',
  'isolation_window': None,
  'last_modified': '2016-06-22T10:38:51',
  'lcms_run': {'acquisition_time': None,
  'creation_time': '2016-02-10T17:08:10',
  'description': u'20160119_KZ_HILIC_Avena_exu_uptake 20160119_pHILIC___POS_MSMS_KZ__KBL_QCMix_V2_______Run80_2.mzML',
  'experiment': u'20160119_KZ_HILIC_Avena_exu_uptake',
  'hdf5_file': u'/global/project/projectdirs/metatlas/raw_data/katezh/20160119_KZ_HILIC_Avena_exu_uptake/20160119_pHILIC___POS_MSMS_KZ__KBL_QCMix_V2_______Run80_2.h5',
  'head_id': u'0697d7b610f6420a8472201ce542bd16',
  'injection_volume': None,
  'injection_volume_units': 'uL',
  'last_modified': '2016-02-10T19:13:56',
  'method': None,
  'mzml_file': u'/global/project/projectdirs/metatlas/raw_data/katezh/20160119_KZ_HILIC_Avena_exu_uptake/20160119_pHILIC___POS_MSMS_KZ__KBL_QCMix_V2_______Run

In [9]:
cids = metob.retrieve('CompoundIdentifications',username='*',unique_id='213a332a08814b4881e4eb45fc42d7b7')
cids[-1].compound[-1].inchi_key

u'JVTAAEKCZFNVCJ-UHFFFAOYSA-N'

In [13]:
to_export = []
frag_spectra = []
frag_file = []
for c in cids:
    if c.frag_references:
        frag_spectra.append([(m.mz, m.intensity) for m in c.frag_references[0].mz_intensities])
        to_export.append(c)
df = metob.to_dataframe(to_export)
df['fragmentation_spectra'] = pd.Series(frag_spectra)

AttributeError: 'FragmentationReference' object has no attribute 'frag_references'

In [ ]:
df.head()

In [ ]:
import ast
def serialize_metatlas_reference_ids(row,object_name):
    c = ast.literal_eval(row[object_name])
    if c:
        c = c[0]
    else:
        c = ''
    return c

df['unique_id_Compounds'] = df.apply(serialize_metatlas_reference_ids,axis=1,object_name = 'compound')
df['unique_id_FragmentationReferences'] = df.apply(serialize_metatlas_reference_ids,axis=1,object_name = 'frag_references')
df['unique_id_MzReferences'] = df.apply(serialize_metatlas_reference_ids,axis=1,object_name = 'mz_references')
df['unique_id_RtReferences'] = df.apply(serialize_metatlas_reference_ids,axis=1,object_name = 'rt_references')
df.drop(['compound','frag_references','mz_references','rt_references','intensity_references','unique_id','description','identification_grade'], axis=1, inplace=True)
df.head()

In [ ]:
my_tables = ['unique_id_Compounds','unique_id_FragmentationReferences','unique_id_MzReferences','unique_id_RtReferences']
table_holder = {}
for tab in my_tables:
    print tab
    obj_list = []
    for u_val in df[tab].unique():
        obj_retrieve = metob.retrieve(tab.split('_')[-1],unique_id = u_val,username = '*')
        if obj_retrieve:
            obj_list.append(obj_retrieve[0])
#         else:
#             print tab,u_val,'not found'
    table_holder[tab] = metob.to_dataframe(obj_list)
    table_holder[tab].rename(columns={'unique_id': tab}, inplace=True)
    print table_holder[tab].shape


In [ ]:
temp_df = pd.merge(left = df,
                   right = table_holder[my_tables[0]],
                   left_on='unique_id_Compounds',
                   right_on='unique_id_Compounds',
                   suffixes = ('_CompoundIdentifications','_Compounds'),
                   how='left')
# temp_df.drop_duplicates(inplace=True)

for tab in my_tables[1:]:
    print tab
    temp_df = pd.merge(left = temp_df,
                       right = table_holder[tab],
                       left_on=tab,
                       right_on=tab,
                       suffixes = ('_CompoundIdentifications','_'+tab.split('_')[-1]),
                       how='left')
#     temp_df.drop_duplicates(inplace=True)
temp_df.rename(columns={'lcms_run_CompoundIdentifications': 'lcms_run_FragmentationReferences'}, inplace=True)
temp_df.rename(columns={'lcms_run': 'lcms_run_RtReferences'}, inplace=True)

print df.shape,temp_df.shape
temp_df.head()



In [ ]:
[c for c in temp_df.columns if 'lcms' in c]

In [ ]:
#Get name, experiment, and method
for i,row in temp_df.iterrows():
    for file_ref in [c for c in temp_df.columns if 'lcms' in c]:
    #     name,experiment,mzml_file
        if not pd.isnull(row[file_ref]):
            f = metob.retrieve('Lcmsruns',unique_id = row[file_ref], username = '*')
            if f:
                temp_df.loc[i,file_ref.split('_')[-1]+'_name'] = f[0].name
                temp_df.loc[i,file_ref.split('_')[-1]+'_experiment'] = f[0].experiment
                temp_df.loc[i,file_ref.split('_')[-1]+'_mzml_file'] = f[0].mzml_file
                temp_df.loc[i,file_ref.split('_')[-1]+'_method'] = f[0].method
                
    #             files['experiment'].append(f[0].experiment)
    #             files['mzml_file'].append(f[0].mzml_file)
    #         else:
    #             files['name'].append('')
    #             files['experiment'].append('')
    #             files['mzml_file'].append('')
temp_df.head()

In [ ]:
temp_df[temp_df.name_Compounds.str.contains('Salicylate')]

In [ ]:
temp_df.to_csv('/global/homes/b/bpb/Downloads/references_out.tab',sep='\t')

In [ ]:
# remove_list = ['ISAKRJDGNUQOIC-UHFFFAOYSA-N']

In [ ]:
terms = ['rt_peak_6550_ZIC-HILIC',
'rt_min_6550_ZIC-HILIC',
'rt_max_6550_ZIC-HILIC',
'rt_peak_QE139_ZIC-HILIC',
'rt_min_QE139_ZIC-HILIC',
'rt_max_QE139_ZIC-HILIC',
'rt_peak_QE144_ZIC-HILIC',
'rt_min_QE144_ZIC-HILIC',
'rt_max_QE144_ZIC-HILIC',
'rt_peak_QE119_ZIC-HILIC',
'rt_min_QE119_ZIC-HILIC',
'rt_max_QE119_ZIC-HILIC',
'rt_peak_6550_pHILIC',
'rt_min_6550_pHILIC',
'rt_max_6550_pHILIC',
'rt_peak_6550_pHILIC-noguard',
'rt_peak_QE139_pHILIC',
'rt_min_QE139_pHILIC',
'rt_max_QE139_pHILIC',
'RT_6550_RP',
'RT_QE139_RP',
        'RT_QE144_RP',
        'RT_QE119_RP']

sorted(list(set(['_'.join(t.split('_')[-2:])+'_0.5min_25ppm_1000counts' for t in terms])))

In [ ]:
to_remove = []
for c in cids:
    if c.frag_references:
        if c.compound:
            if c.compound[0].inchi_key in remove_list:
                to_remove.append(f)
                print c.unique_id,c.compound[0].inchi_key,c.username
#                 metob.remove('CompoundIdentifications',unique_id = c.unique_id,username = c.username,_override = True)

In [ ]:
# metob.remove('CompoundIdentifications',unique_id = '3f82a1e296ca4d1cb61b7ac6cf84f6d3',username = 'rlau',_override = True)

In [ ]:
len(to_remove)

In [ ]:
# query = 'SELECT DISTINCT experiment,creation_time,username FROM lcmsruns where creation_time >= UNIX_TIMESTAMP(DATE_SUB(CURDATE(), INTERVAL %d DAY))'%num_days
query = 'SELECT * from compoundidentifications_frag_references where unique_id is 9932583619ed4d92beba17a50f32bd40'#  experiment,creation_time,username FROM lcmsruns where creation_time >= UNIX_TIMESTAMP(DATE_SUB(CURDATE(), INTERVAL %d DAY))'%num_days

entries = [e for e in metob.database.query(query)]
len(entries)

In [ ]:
fr = metob.retrieve('FragmentationReferences',username='*')

In [ ]:
mz = []
for f in fr:
    mz.append(f.precursor_mz)
for s in sorted(list(set(mz))):
    print '%5.6f'%s

In [ ]:
cid = metob.retrieve('CompoundIdentifications',username = '*')

In [ ]:
dir(cid[0])

In [ ]:
for c in cid:
    if c.frag_references:
        if abs(c.frag_references[0].precursor_mz - 128.035324) < 0.01:
            print c.name
            print c.compound
            c.compound = metob.retrieve('Compounds',inchi_key = 'ODHCTXKNWHHXJC-VKHMYHEASA-N')
            metob.store(c)

In [ ]:
compounds = metob.retrieve('Compounds',synonyms='%pyroglutamic%')
for c in compounds:
    print c.name
    print c.inchi_key
    print ''